In [1]:
import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from torch.utils.data import Dataset
import os
import pandas as pd
import requests
from tqdm.auto import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, matthews_corrcoef
from sklearn.metrics import confusion_matrix
import re
import warnings

os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_NOTEBOOK_NAME"] = "ProtBert-CPPSet3.ipynb"

warnings.filterwarnings("ignore")

In [2]:
model_name = '../Rostlab/prot_bert'

In [3]:
class DeepLocDataset(Dataset):
    def __init__(self, split="train", tokenizer_name=model_name, max_length=80):
        self.datasetFolderPath = '../dataset/'
        self.trainFilePath = os.path.join(self.datasetFolderPath, 'CPPSet3-train.csv')
        self.testFilePath = os.path.join(self.datasetFolderPath, 'CPPSet3-test.csv')
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, do_lower_case=False)
        if split=="train":
            self.seqs, self.labels = self.load_dataset(self.trainFilePath)
        else:
            self.seqs, self.labels = self.load_dataset(self.testFilePath)
        self.max_length = max_length
        
    def load_dataset(self,path):
        df = pd.read_csv(path,names=['input','labels'],skiprows=1)
        seq = list(df['input'])
        seq = [' '.join(i) for i in seq]
        label = list(df['labels'])
        assert len(seq) == len(label)
        return seq, label

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        seq = " ".join("".join(self.seqs[idx].split()))
        seq = re.sub(r"[UZOB]", "X", seq)
        seq_ids = self.tokenizer(seq, truncation=True, padding='max_length', max_length=self.max_length)
        sample = {key: torch.tensor(val) for key, val in seq_ids.items()}
        sample['labels'] = torch.tensor(self.labels[idx])
        return sample

In [4]:
train_dataset = DeepLocDataset(split="train", tokenizer_name=model_name, max_length=80)
test_dataset = DeepLocDataset(split="test", tokenizer_name=model_name, max_length=80)

In [5]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    specificity = tn / (tn+fp)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'specificity': specificity,
        'f1': f1,
        'mcc' : matthews_corrcoef(labels, preds)
    }

In [6]:
def model_init():
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    for param in model.parameters(): param.data = param.data.contiguous()
    return model

In [7]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    output_dir='../results',
    num_train_epochs=10,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='../logs',
    logging_steps=2,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=30,
    fp16=True,
    fp16_opt_level="02",
    run_name="ProBert-BFD-MS",
    seed=42
)

# 26

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics = compute_metrics,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../Rostlab/prot_bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../Rostlab/prot_bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Specificity,F1,Mcc
0,No log,0.664176,0.932935,0.000000,0.000000,1.000000,0.000000,0.000000
1,0.693300,0.672204,0.932935,0.500000,0.012739,0.999084,0.024845,0.071606
2,0.689600,0.638558,0.914139,0.408333,0.624204,0.934982,0.493703,0.461119
3,0.663100,0.546858,0.941051,0.570370,0.490446,0.973443,0.527397,0.497761
4,0.613800,0.663192,0.635199,0.146910,0.923567,0.614469,0.253497,0.272533
5,0.542700,0.466026,0.871850,0.311346,0.751592,0.880495,0.440299,0.429224


TrainOutput(global_step=10, training_loss=0.6405044078826905, metrics={'train_runtime': 229.7866, 'train_samples_per_second': 49.916, 'train_steps_per_second': 0.044, 'total_flos': 1079525462798400.0, 'train_loss': 0.6405044078826905, 'epoch': 5.172413793103448})

In [8]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.46602606773376465,
 'eval_accuracy': 0.8718496369073045,
 'eval_precision': 0.3113456464379947,
 'eval_recall': 0.7515923566878981,
 'eval_specificity': 0.8804945054945055,
 'eval_f1': 0.4402985074626866,
 'eval_mcc': 0.42922380521278286,
 'eval_runtime': 19.1917,
 'eval_samples_per_second': 121.98,
 'eval_steps_per_second': 6.148,
 'epoch': 5.172413793103448}

In [9]:
trainer.save_model('../model/ProtBert_CPPSet3.pt')

In [10]:
result = trainer.predict(test_dataset).predictions
golden = trainer.predict(test_dataset).label_ids
result = torch.argmax(torch.tensor(result), -1)

In [11]:
torch.save(result, '../model/CPPSet3_result1.pt')

In [12]:
torch.save(golden, '../model/CPPSet3_golden1.pt')